## Модуль 3, домашнее задание
#### Автор: Вячеслав Барков

In [1]:
%load_ext sql
%config SqlMagic.displaycon = False
%config SqlMagic.displaylimit = 10

In [2]:
import json
import sqlalchemy
import pandas as pd

In [3]:
with open('.secret') as f:
    data = json.load(f)
    user = data['user']
    password = data['password']
    server = data['server']
    db = data['db']

In [4]:
engine = sqlalchemy.create_engine(f'postgresql://{user}:{password}@{server}/{db}')

In [5]:
%sql postgresql://$user:$password@$server/$db

In [6]:
%%sql

SET
    search_path
TO
    shipping;

Done.


[]

## Задание 1.


Напишите код DDL для создания представления (VIEW), которое будет объединять в себе все значения атрибутов по всем поставкам. Это представление должно включать в себя в денормализованной форме все данные по поставке из нашей БД. Одна строка – одна поставка. Идентификаторы записей из таблиц, при помощи которых мы связываем таблицы соединениями (JOIN) в представление включать НЕ НАДО. Только фактические значения. 

In [7]:
%%sql

DROP VIEW IF EXISTS shipping.V_shipping_complete;

Done.


[]

**Решение:** Создадим представление с данными о каждой поставке, данными о городе каждой поставки, данными о заказчике, данными о городе заказчика, данными о водителе, данными о городе водителя, данными о грузовике.

In [8]:
%%sql

CREATE OR REPLACE VIEW shipping.V_shipping_complete AS
SELECT 
    s.ship_date,
    s.weight,
    cs.city_name ship_city_name,
    cs.state ship_state,
    cs.population ship_city_population,
    cs.area ship_city_area,
    c.cust_name,
    c.annual_revenue cust_annual_rev,
    c.cust_type,
    c.address cust_address,
    c.zip cust_zip,
    c.phone cust_phone,
    cc.city_name cust_city_name,
    cc.state cust_state,
    cc.population cust_city_population,
    cc.area cust_city_area,
    d.first_name driver_first_name,
    d.last_name driver_last_name,
    d.address driver_address,
    d.zip_code driver_zip_code,
    d.phone driver_phone,
    cd.city_name driver_city_name,
    cd.state driver_state,
    cd.population driver_city_population,
    cd.area driver_city_area,
    t.make truck_make,
    t.model_year truck_model_year
FROM 
    shipment s
    LEFT JOIN customer c ON c.cust_id = s.cust_id
    LEFT JOIN driver d ON d.driver_id = s.driver_id
    LEFT JOIN truck t ON t.truck_id = s.truck_id
    LEFT JOIN city cs ON cs.city_id = s.city_id    -- Город доставки
    LEFT JOIN city cd ON cd.city_id = d.city_id    -- Город водителя
    LEFT JOIN city cc ON cc.city_id = c.city_id;   -- Город клиента

Done.


[]

## Задание 2.


Ваш условный бизнес-заказчик просит модифицировать нашу схему и добавить таблицу, где будет храниться информация о доставленном в ходе поставки грузе.  Мы характеризуем груз следующим образом:
- Название груза – некое наименование груза
- Вид груза (одно из: Оборудование, Продукты питания, Прочие товары) 
- Вес груза по накладной,  кг
- Доставлено до получателя, кг (мы предполагаем, что часть груза может быть повреждена в ходе доставки) 
- Доля повреждений ( % товара не принятого получателем вследствие повреждений).

Типы данных колонок необходимо определить самостоятельно. Также необходимо связать строки таблицы грузов с поставками, чтобы мы могли потом анализировать, какие грузы доставлялись в рамках одной поставки. Дополните код таблицы Shipment и вашего прототипа таблицы соответствующими полями 


**Решение:** Будем использовать первичный ключ cargo_id, а также вторичный ключ ship_id для связи таблицы грузов с таблицей поставок. Закодируем вид груза и создадим отдельную таблицу для его расшифровки. Необходимо уточнить у заказчика источник поля доли повреждений. Исходя из предоставленного задания, создадим генерируемое поле для расчета доли повреждений.

In [9]:
%%sql

DROP TABLE IF EXISTS shipping.cargo, shipping.cargo_types;

Done.


[]

In [10]:
%%sql

CREATE TABLE shipping.cargo (
    cargo_id bigint PRIMARY KEY,
    ship_id bigint,
    cargo_name text,
    type_id smallint,
    weight_init NUMERIC CHECK (weight_init > 0),
    weight_last NUMERIC CHECK (weight_last >= 0),
    damage_rate decimal(6,3) GENERATED ALWAYS AS ((1 - weight_last/weight_init) * 100) STORED 
);

Done.


[]

In [11]:
%%sql

CREATE TABLE shipping.cargo_types (
    type_id smallint PRIMARY KEY,
    name text   
);

Done.


[]

In [12]:
%%sql

INSERT INTO
    shipping.cargo_types
VALUES
    (1, 'Оборудование'),
    (2, 'Продукты питания'),
    (3, 'Прочие товары');

3 rows affected.


[]

In [13]:
%%sql

INSERT INTO
    cargo
VALUES
    (1006,    1000,    'Модем',    1,    100,    98.00), 
    (1007,    1000,    'Картофель',    2,    100.05,    90.00);

2 rows affected.


[]

In [14]:
%%sql

SELECT
    *
FROM
    shipping.cargo c
    LEFT JOIN cargo_types t ON t.type_id = c.type_id;

2 rows affected.


cargo_id,ship_id,cargo_name,type_id,weight_init,weight_last,damage_rate,type_id_1,name
1006,1000,Модем,1,100,98.00,2.000,1,Оборудование
1007,1000,Картофель,2,100.05,90.00,10.045,2,Продукты питания


## Задание 3.


Создайте код таблицы для хранения данных из следующего источника https://www.kaggle.com/nobelfoundation/nobel-laureates . Это файл CSV, который мы условно хотим разместить в Stage Area (схема хранения сырых данных, которые далее предполагается использовать при создании витрин).  Исследуйте источник, задайте правильные типы данных, чтобы мы могли разместить в нашей таблице эти данные AS IS, без изменений.

**Решение:** Выберем типы данных с условием импорта предоставленного файла. Учтем особенности предоставленного csv. 
- Поле "Prize Share" содержит дробь в математической записи, поэтому сохраним его как текст. В дальнейшем, в зависимости от целей заказчика, можно преобразовать его в десятичную дробь или сохранить отдельно числитель и знаменатель как целочисленные числа.
- Поле "Birth Date" содержит дату, но присутствуют данные неверного формата, например "1943-00-00", поэтому вместо типа date пока используем тип varchar(10).
- Поле "Sex" в дальнейшем можно закождировать и создать отдельный словарь

В дальнейшем, с учетом изменения данных, типы можно пересмотреть.

In [15]:
!explorer .

In [16]:
%%sql

CREATE SCHEMA IF NOT EXISTS nobel;

Done.


[]

In [17]:
%%sql

DROP TABLE IF EXISTS nobel.nobel_laureates;

Done.


[]

In [18]:
%%sql

CREATE TABLE nobel.nobel_laureates (
    prize_year SMALLINT,
    category TEXT,
    prize TEXT,
    motivation TEXT,
    prize_share text,
    laureate_id int,
    laureate_type TEXT,
    full_name TEXT,
    birth_date varchar(10),
    birth_city TEXT,
    birth_country TEXT,
    sex TEXT,
    organization_name TEXT,
    organization_city TEXT,
    organization_country TEXT,
    death_date date,
    death_city TEXT,
    death_country TEXT
);

Done.


[]

In [19]:
%%sql

set datestyle = 'ISO, YMD';

Done.


[]

In [20]:
%%sql

\copy
    nobel.nobel_laureates
FROM
    'data/nobel_archive.csv' 
DELIMITER ',' 
CSV HEADER;

0 rows affected.


[]

In [22]:
%%sql

SELECT
    n.prize_year,
    n.category,
    n.prize,
    n.motivation,
    n.prize_share,
    n.laureate_id,
    n.full_name,
    n.birth_date,
    n.birth_city
FROM
    nobel.nobel_laureates n
LIMIT 3;

3 rows affected.


prize_year,category,prize,motivation,prize_share,laureate_id,full_name,birth_date,birth_city
1901,Chemistry,The Nobel Prize in Chemistry 1901,"""in recognition of the extraordinary services he has rendered by the discovery of the laws of chemical dynamics and osmotic pressure in solutions""",1/1,160,Jacobus Henricus van 't Hoff,1852-08-30,Rotterdam
1901,Literature,The Nobel Prize in Literature 1901,"""in special recognition of his poetic composition, which gives evidence of lofty idealism, artistic perfection and a rare combination of the qualities of both heart and intellect""",1/1,569,Sully Prudhomme,1839-03-16,Paris
1901,Medicine,The Nobel Prize in Physiology or Medicine 1901,"""for his work on serum therapy, especially its application against diphtheria, by which he has opened a new road in the domain of medical science and thereby placed in the hands of the physician a victorious weapon against illness and deaths""",1/1,293,Emil Adolf von Behring,1854-03-15,Hansdorf (Lawice)
